In [ ]:
import pandas as pd
import numpy as np
import os
import re
from nltk import word_tokenize,sent_tokenize,pos_tag_sents,pos_tag
import time

## 1. Read data.

In [2]:
DATADIR=os.getenv('DATADIR')
DATAPATH = os.path.join(DATADIR, 'labelled_level2.csv.gz')

In [3]:
data = pd.read_csv(DATAPATH,compression='gzip')
data.shape

/Users/felisialoukou/.pyenv/versions/3.4.6/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(172916, 21)

In [4]:
data.head()

,base_path,content_id,description,details,document_type,first_published_at,locale,primary_publishing_organisation,publishing_app,title,...,body,combined_text,taxon_id,taxon_base_path,taxon_name,level1taxon,level2taxon,level3taxon,level4taxon,level5taxon
0,/student-finance-register-login,e57daef4-5eb5-431c-b0ad-14119ab0355f,your student finance online account - check pa...,{'other_ways_to_apply': '<p>If you can’t apply...,transaction,2016-02-29T09:24:10.000+00:00,en,NaN,publisher,student finance login,...,sign in or register for student finance online...,student finance login your student finance onl...,64aa6eec-48b5-481d-9131-9c8b6326eea1,/education/student-grants-bursaries-scholarships,"Student grants, bursaries and scholarships","Education, training and skills",Funding and finance for students,"Student grants, bursaries and scholarships",NaN,NaN
1,/student-finance-calculator,434b6eb5-33c8-4300-aba3-f5ead58600b8,student finance calculator - get a quick estim...,{'transaction_start_link': '/student-finance-c...,transaction,2016-02-29T09:24:10.000+00:00,en,NaN,smartanswers,student finance calculator,...,this calculator is for students from england o...,student finance calculator student finance cal...,64aa6eec-48b5-481d-9131-9c8b6326eea1,/education/student-grants-bursaries-scholarships,"Student grants, bursaries and scholarships","Education, training and skills",Funding and finance for students,"Student grants, bursaries and scholarships",NaN,NaN
2,/apply-online-for-student-finance,83155b50-418e-437c-9389-cf0e1302900f,apply online for student finance - online appl...,"{'other_ways_to_apply': '<h2 id=""apply-by-post...",transaction,2016-02-29T09:24:10.000+00:00,en,NaN,publisher,apply online for student finance,...,if you’re a student from england you can apply...,apply online for student finance apply online ...,64aa6eec-48b5-481d-9131-9c8b6326eea1,/education/student-grants-bursaries-scholarships,"Student grants, bursaries and scholarships","Education, training and skills",Funding and finance for students,"Student grants, bursaries and scholarships",NaN,NaN
3,/student-finance,d38bafd3-2c46-4be2-b50b-50c2ba7d30ed,student finance - student loans or student gra...,{'parts': [{'body': '<p>You may be able to bor...,guide,2016-02-29T09:24:10.000+00:00,en,NaN,publisher,student finance,...,you may be able to borrow money to help pay fo...,student finance student finance - student loan...,64aa6eec-48b5-481d-9131-9c8b6326eea1,/education/student-grants-bursaries-scholarships,"Student grants, bursaries and scholarships","Education, training and skills",Funding and finance for students,"Student grants, bursaries and scholarships",NaN,NaN
4,/apply-for-student-finance,06017464-d3e2-4a4c-8bef-250eff0de7e4,how to apply for student finance - when to app...,"{'parts': [{'body': '<h2 id=""new-students-from...",guide,2016-02-29T09:24:10.000+00:00,en,NaN,publisher,student finance: how to apply,...,how to apply new students from england most fu...,student finance: how to apply how to apply for...,64aa6eec-48b5-481d-9131-9c8b6326eea1,/education/student-grants-bursaries-scholarships,"Student grants, bursaries and scholarships","Education, training and skills",Funding and finance for students,"Student grants, bursaries and scholarships",NaN,NaN


In [5]:
# /student-finance-register-login
def text_at_index(column,index):
    if isinstance(column,str):
        print(data[column].iloc[index])
    else:
        for c in column:
            print(data[c].iloc[index])

In [6]:
text_fields = ['title','description', 'body']
reduced_fields = ['reduced_title','reduced_description', 'reduced_combined_text']

In [7]:
text_at_index(text_fields,0)

student finance login
your student finance online account - check payment dates track an application change details reset password find customer reference numbers
sign in or register for student finance online. you can: view your statements and letters from student finance england track an existing application check when your payments are due update some of your personal or application details reset your password or find your customer reference number apply for finance as a new or continuing student if you’re a parent or partner supporting a student’s application there’s a different way to submit or update information about your income . you’ll be given a customer reference number ( crn ) and asked to create a password and secret answer when you set up your account. you need these each time you sign in. they’re sent by post if you don’t apply online. if you can’t apply online use the form finder to get the forms you need. you can call student finance england if you want to apply online

student finance login
your student finance online account - check payment dates track an application change details reset password find customer reference numbers

### 1.1 UTILS.

In [8]:
data.columns

Index(['base_path', 'content_id', 'description', 'details', 'document_type',
       'first_published_at', 'locale', 'primary_publishing_organisation',
       'publishing_app', 'title', 'document_type_gp', 'body', 'combined_text',
       'taxon_id', 'taxon_base_path', 'taxon_name', 'level1taxon',
       'level2taxon', 'level3taxon', 'level4taxon', 'level5taxon'],
      dtype='object')

In [9]:
def build_index(x):
    index_dict = {}
    index_dict['index'] = 0
    for i,elem in enumerate(x):
        index_dict[elem] = i+1
    return index_dict

In [10]:
ind = build_index(data.columns)
print(ind['combined_text'])

13


## 2. Filter target Part of Speech tags and punctuation.


In [11]:
def is_junk(word):
    return len(word) < 2

def is_aux(word):
    m = re.match("\\b[iI]s\\b|\\b[aA](m|re)\\b|\\b[bB](een|e)\\b|\\b[hH](ave|as)\\b|\\b[wW](as|ere|ill|ould)\\b",word)
    return bool(m)

In [12]:
test = "been"
print(is_aux(test))
test2 = ""
print(is_junk(test2))

True
True


In [13]:
def reduce_text(x):
    keep_text =""
    sent_pos = pos_tag(word_tokenize(x))
    for word,tag in sent_pos:
        if not is_junk(word) and (tag.startswith("NN") or (tag.startswith("VB") and 
                                                           not is_aux(word)) or tag.startswith("JJ")):
            keep_text += word + " "
    return keep_text.strip()

### 2.1 Filter all text fields.

In [ ]:
# 'title', 'body',
text_fields = ['title','description','combined_text']
reduced_fields = []
for field in text_fields:
    print("WORKON: "+field+"\n"+"STARTED: "+time.strftime("%H:%M:%S %d-%M-%Y"))
    name = 'reduced_'+field
    print("PRODUCING:",name)
    reduced_fields.append(name)
    print("ONGOING FIELD LIST:",reduced_fields)
    data[name] = data[field].map(reduce_text)
    print("FINISHED:",time.strftime("%H:%M:%S %d-%M-%Y"))

WORKON: title
STARTED: 15:26:41 12-26-2018
PRODUCING: reduced_title
ONGOING FIELD LIST: ['reduced_title']
FINISHED: 15:28:44 12-28-2018
WORKON: description
STARTED: 15:28:44 12-28-2018
PRODUCING: reduced_description
ONGOING FIELD LIST: ['reduced_title', 'reduced_description']
FINISHED: 15:32:05 12-32-2018
WORKON: combined_text
STARTED: 15:32:05 12-32-2018
PRODUCING: reduced_combined_text
ONGOING FIELD LIST: ['reduced_title', 'reduced_description', 'reduced_combined_text']


### 2.2 Rename and drop unfiltered columns.

In [ ]:
data.columns

In [ ]:
data.head()

In [ ]:
print(data['title'].iloc[1])
print(data['reduced_title'].iloc[1])

In [ ]:
print(text_fields,reduced_fields)

In [ ]:
for field,reduced_field in zip(text_fields,reduced_fields):
    print(field,reduced_field)
    data[field] = data[reduced_field] 

In [ ]:
data.drop(['reduced_'+field for field in text_fields],axis=1,inplace=True)

In [ ]:
data.head()

## 3. Write to file.

In [ ]:
file = 'labelled_level2_filtered_all_beta.csv.gz'
OUTPUT= os.path.join(DATADIR, file)
data.to_csv(OUTPUT,compression='gzip')